In [5]:
import sys
sys.path.append('..')
from dotenv import load_dotenv

load_dotenv()

from src.vector import FAISSVectorStore, EmbeddingVector, create_file_content_map, OpenAIEmbeddingProvider, EmbeddingConfig
print(FAISSVectorStore, EmbeddingVector, create_file_content_map, OpenAIEmbeddingProvider)

<class 'src.vector.vector.FAISSVectorStore'> <class 'src.vector.vector.EmbeddingVector'> <function create_file_content_map at 0x0000024AFCF8AF20> <class 'src.vector.embedding.OpenAIEmbeddingProvider'>


In [3]:
import os
file_map = create_file_content_map(os.path.join("..", "src"), accept_patterns=["*.py"])

Found 16 valid files


In [ ]:

config = EmbeddingConfig(model_name="text-embedding-3-small",
                         api_key=os.getenv("OPENAI_API_KEY"))
provider = OpenAIEmbeddingProvider(config=config)

embeddings = await provider.embed_documents(list(file_map.values()))






In [13]:
emb_vectors = [EmbeddingVector(id=file_path,
                               vector=embedding,
                               metadata={"source": file_path},
                               text=file_map[file_path]) for file_path, embedding in zip(file_map.keys(), embeddings)]


In [15]:
f_store = FAISSVectorStore(dimension=1536, index_path="faiss_index.bin")
f_store.add_vectors(emb_vectors)


In [19]:
query_vector = await provider.embed_query("how are we implementing the vector database")
query_results = f_store.query(query_vector=query_vector, k=5)


for result in query_results:
    print(result.metadata["source"])
    print(result.text)



INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


C:\Users\Utkarsh Priyadarshi\WebstormProjects\gep\src\vector\vector.py
import os
import faiss
import numpy as np

from abc import ABC, abstractmethod
from dataclasses import dataclass
from enum import Enum, auto
from typing import Any, Optional

from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

from langchain_core.embeddings import Embeddings
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma, FAISS, PGVector, Pinecone


@dataclass
class EmbeddingVector:
    """Represents an embedding vector with its metadata."""
    id: str
    vector: list[float]
    metadata: dict[str, Any]
    text: str


@dataclass
class SearchResult:
    """Represents a search result from the vector store."""
    text: str
    metadata: dict[str, Any]
    score: float
    vector_id: str


class VectorStoreType(Enum):
    """
    Enum for the type of vector store.
    """
    CHROMA = auto()
    PGVECTOR = auto()
    FAISS = a